`Done - Initiatives, keeping only scope counters`

Note that this dataset has already been converted to firm_year, the only missing part is the detail on which scopes the incentives apply. To this end, I will just keep the three relevant columns, groupby id and year, and then sum.

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns

In [25]:
data = pd.read_stata("../../data/CDP Cleaning/cleaned outputs/cdp_initiative.dta")

In [26]:
data.shape

(113003, 27)

In [27]:
data.head()

,id,year,cdp_ini_type_detail_f,cdp_ini_type_f,cdp_ini_co2_c,cdp_ini_ghgscope_f,cdp_ini_disclosure_f,cdp_ini_monetarysaving_c,cdp_ini_investmentrequired_c,cdp_ini_payback_f,...,scope3,cdp_ini_subtype_f,reporting_boundary,companyname,country,industry14,isin,ticker,industry46,cdpdate
0,44,2011.0,• we have requested data from portfolio compan...,company policy or behavioral change,NaN,,,NaN,NaN,,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,NaT
1,44,2013.0,other,other,901.50,,,9303.00,1550.5,1-3 years,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2013-06-18
2,44,2013.0,3i are undertaking an accreditation exercise w...,energy efficiency in production processes,901.50,,,9303.00,1550.5,4-10 years,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2013-06-18
3,44,2014.0,reducing travel associated carbon emissions. ...,transportation,39.67,,,105766.11,0.0,1-3 years,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2014-06-26
4,44,2014.0,reducing electricty use with our buildings. ...,energy efficiency in buildings,10.31,,,3505.33,0.0,1-3 years,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2014-06-26


In [28]:
data.columns

Index(['id', 'year', 'cdp_ini_type_detail_f', 'cdp_ini_type_f',
       'cdp_ini_co2_c', 'cdp_ini_ghgscope_f', 'cdp_ini_disclosure_f',
       'cdp_ini_monetarysaving_c', 'cdp_ini_investmentrequired_c',
       'cdp_ini_payback_f', 'cdp_ini_lifetime_f', 'cdp_ini_comment_s',
       'cdp_ini_comment_lower_s', 'cdp_currency', 'currency_ex', 'scope1',
       'scope2', 'scope3', 'cdp_ini_subtype_f', 'reporting_boundary',
       'companyname', 'country', 'industry14', 'isin', 'ticker', 'industry46',
       'cdpdate'],
      dtype='object')

In [29]:
data.drop(columns=['reporting_boundary',
       'companyname', 'country', 'industry14', 'isin', 'ticker', 'industry46',
       'cdpdate', 'cdp_currency', 'currency_ex','cdp_ini_comment_s', 'cdp_ini_comment_lower_s', 'cdp_ini_ghgscope_f', 'cdp_ini_type_detail_f'], inplace=True)

In [30]:
data.columns

Index(['id', 'year', 'cdp_ini_type_f', 'cdp_ini_co2_c', 'cdp_ini_disclosure_f',
       'cdp_ini_monetarysaving_c', 'cdp_ini_investmentrequired_c',
       'cdp_ini_payback_f', 'cdp_ini_lifetime_f', 'scope1', 'scope2', 'scope3',
       'cdp_ini_subtype_f'],
      dtype='object')

In [31]:
data.sample(5)

,id,year,cdp_ini_type_f,cdp_ini_co2_c,cdp_ini_disclosure_f,cdp_ini_monetarysaving_c,cdp_ini_investmentrequired_c,cdp_ini_payback_f,cdp_ini_lifetime_f,scope1,scope2,scope3,cdp_ini_subtype_f
35033,9134,2014.0,other,9000.00,,201080.589603,4.857019e+06,21-25 years,50 years,0.0,0.0,0.0,
61402,17069,2021.0,energy efficiency in buildings,48.27,Voluntary,12768.000000,2.171290e+05,16-20 years,16-20 years,0.0,1.0,0.0,hvac
32523,8587,2021.0,energy efficiency in buildings,1335.00,Voluntary,333723.000000,2.597730e+06,4-10 years,21-30 years,1.0,1.0,0.0,maintenance program
57455,15980,2020.0,other,128873.00,Voluntary,0.000000,4.308716e+04,No payback,<1 year,1.0,1.0,0.0,
7648,1362,2022.0,low-carbon energy,37544.00,Voluntary,0.000000,0.000000e+00,No payback,Ongoing,0.0,1.0,0.0,low-carbon electricity mix


In [32]:
for column in data.columns:
    print(column + ' ' + str(len(data[column].unique())))

id 10963
year 12
cdp_ini_type_f 11
cdp_ini_co2_c 24775
cdp_ini_disclosure_f 8
cdp_ini_monetarysaving_c 47663
cdp_ini_investmentrequired_c 39445
cdp_ini_payback_f 20
cdp_ini_lifetime_f 1834
scope1 2
scope2 2
scope3 2
cdp_ini_subtype_f 49


In [33]:
data = data[['id', 'year', 'scope1', 'scope2', 'scope3']]

In [34]:
data = data.groupby(['id', 'year']).sum().reset_index()

In [35]:
data['scope3'].sort_values(ascending=False)

1050     42.0
13673    28.0
4649     24.0
9485     22.0
9486     21.0
         ... 
11767     0.0
11766     0.0
11765     0.0
11764     0.0
32205     0.0
Name: scope3, Length: 32206, dtype: float32

In [36]:
data['scope1'].value_counts()

scope1
0.0     20811
1.0      5495
2.0      2506
3.0      1327
4.0       784
5.0       471
6.0       279
7.0       149
8.0       107
9.0        74
10.0       48
11.0       41
12.0       18
13.0       16
14.0       15
15.0        8
18.0        8
19.0        8
20.0        7
17.0        5
16.0        5
25.0        3
23.0        3
21.0        2
27.0        2
29.0        2
30.0        1
28.0        1
22.0        1
32.0        1
24.0        1
35.0        1
31.0        1
45.0        1
38.0        1
74.0        1
37.0        1
40.0        1
Name: count, dtype: int64

In [37]:
# taking the log1p of the numberical variables for the scope counts
data['scope1'] = np.log1p(data['scope1'])
data['scope2'] = np.log1p(data['scope2'])
data['scope3'] = np.log1p(data['scope3'])

In [38]:
data.isna().sum()

id        0
year      0
scope1    0
scope2    0
scope3    0
dtype: int64

In [39]:
data['scope3'].mean()

0.15625417

In [40]:
data.shape

(32206, 5)

In [41]:
data.columns

Index(['id', 'year', 'scope1', 'scope2', 'scope3'], dtype='object')

In [42]:
data.shape

(32206, 5)

In [43]:
data['initiative_scope1'].value_counts()

KeyError: 'initiative_scope1'

In [ ]:
# rename columns scope 1, 2, 3 to initiative_scope1, 2, 3 binary
data.rename(columns={'scope1': 'initiative_scope1', 'scope2': 'initiative_scope2', 'scope3': 'initiative_scope3'}, inplace=True)

In [ ]:
data.to_csv('../../data/processed/cdp_initiative_processed.csv', index=False)